In [1]:
!git clone https://github.com/rizkydermawan1992/drowsiness-detection.git

Cloning into 'drowsiness-detection'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 1), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [ ]:
!pip3 install -r requirements.txt

In [1]:
%cd drowsiness-detection

/home/jetson/Notebooks/driveralert/drowsiness-detection


In [2]:
%ls

face_landmark_detection.py  README.md
main.py                     shape_predictor_68_face_landmarks.dat


In [2]:
from scipy.spatial import distance as dist
from imutils import face_utils
from threading import Thread
import numpy as np
import dlib
import cv2
import pyfirmata
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import time
from jetcam.usb_camera import USBCamera
import torchvision
import torch
import torch.nn.functional as F
import traitlets

In [3]:
camera = USBCamera(capture_device=0)
#camera.read()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='Dormido', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='Somnoliento', min=0.0, max=0.5, value=0.0, step=0.01, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image_widget, blocked_slider, speed_slider]) ]))

In [4]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear

def mouth_aspect_ratio(mouth):
	
	A = dist.euclidean(mouth[13], mouth[19])
	B = dist.euclidean(mouth[14], mouth[18])
	C = dist.euclidean(mouth[15], mouth[17])

	# compute the mouth aspect ratio
	mar = (A+B+C)/3.0

	# return the eye aspect ratio
	return mar

# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EYE_AR_THRESH = 0.25
MOUTH_AR_THRESH = 35
EYE_AR_CONSEC_FRAMES = 10

# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
COUNTER = 0
ALARM_ON = False

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# start the video stream thread
print("[INFO] starting video stream thread...")



[INFO] loading facial landmark predictor...
[INFO] starting video stream thread...


In [5]:
def update(change):
    global COUNTER
    image = change['new']
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    rects = detector(image, 0)
      
    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shapes = predictor(gray, rect)
        shape = face_utils.shape_to_np(shapes)

        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        mouth    = shape[mStart:mEnd]
        leftEye  = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        mar      = mouth_aspect_ratio(mouth)
        leftEAR  = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0


        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        mouthHull    = cv2.convexHull(mouth)
        leftEyeHull  = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(image, [mouthHull], -1, (0, 255, 0), 1)
        cv2.drawContours(image, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(image, [rightEyeHull], -1, (0, 255, 0), 1)



        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if mar > MOUTH_AR_THRESH or ear < EYE_AR_THRESH :
            COUNTER += 1

            # if the eyes were closed for a sufficient number of
            # then sound the alarm
            if COUNTER >= EYE_AR_CONSEC_FRAMES:               
                blocked_slider.value = 1
        # otherwise, the eye aspect ratio is not below the blink
        # threshold, so reset the counter and alarm
        else:
            COUNTER = 0
            blocked_slider.value = 0

        # draw the computed eye aspect ratio on the frame to help
        # with debugging and setting the correct eye aspect ratio
        # thresholds and frame counters


    #cv2.putText(image, "EAR: {:.2f}".format(ear), (900, 70),
    #cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
   # cv2.putText(image, "MAR: {:.2f}".format(mar), (900, 110),
   # cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    image_widget.value = bgr8_to_jpeg(image)

    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

In [6]:
camera.running = True
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [10]:
camera.unobserve_all()